In [1]:
import pandas as pd
import numpy as np

## Get ids, labels, some road features

In [2]:
accident_df = pd.read_csv("accidents-route.csv")

c:\Users\rahar\.conda\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
accident_df.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [4]:
accident_df.head()

,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
0,Délit de fuite CSR,H7V3A2,2019-07-01,2019-07-02,2019-06-30,2019,6,30,2019-07-01,,...,PDQ3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen
4,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2018.0,BLA,CAMIO,QUE,FERMEE,INTACT,GMC,SAVA,Moyen


In [5]:
infraction_types = accident_df["code-infraction"].unique()
infraction_types

array(['Délit de fuite CSR', 'Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route',
       'Incendie de véhicule en mouvement'], dtype=object)

We will only try to predict collisions.

In [6]:
collisions = ['Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route']

collisions_df = accident_df[accident_df["code-infraction"].isin(collisions)]

In [7]:
print(collisions_df.shape)
collisions_df.head(3)

(24230, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen


In [8]:
collisions_2018 = collisions_df[collisions_df["annee-infraction"].isin([2015,2016,2017,2018,2019])]
print(collisions_2018.shape)
collisions_2018.head(3)

(24230, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen


In [9]:
collisions_2018.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [10]:
from pyexpat import features


features = ["code-infraction","mois-infraction","type-rue","rue","type-rue-2","annee-infraction"]
collisions_2018 = collisions_2018[features].dropna()

In [11]:
collisions_2018.head()

,code-infraction,mois-infraction,type-rue,rue,type-rue-2,annee-infraction
1,Collision matérielle de 2 000 $ et -,6,BOULEV,SAMSON,AVENUE,2019
2,Collision matérielle de 2 000 $ et -,6,BOULEV,SAMSON,AVENUE,2019
3,Collision matérielle de + de 2 000 $,6,BOULEV,CHOMEDEY,BOULEV,2019
4,Collision matérielle de + de 2 000 $,6,BOULEV,CHOMEDEY,BOULEV,2019
6,Collision avec blessé,6,RUE,MAURICE-GAUVIN,BOULEV,2019


We now want to find how many collisions happen on a particular road in a given year and month.

In [39]:
type_rue = collisions_2018["type-rue-2"].unique()
rues = collisions_2018["rue"].unique()

In [14]:
collisions_2018["id"] = collisions_2018["rue"] + "@" + collisions_2018["annee-infraction"].astype(str) + "@" + collisions_2018["mois-infraction"].astype(str)

In [16]:
uniques = collisions_2018["id"].unique()

In [40]:
# count = collisions_2018["id"].value_counts()
_count = collisions_2018.groupby(["id"])["id"].count()


In [41]:
months = [int(i.split("@")[2]) for i in uniques]
year = [int(i.split("@")[1]) for i in uniques]
road = [i.split("@")[0] for i in uniques]
road_type = [collisions_2018[collisions_2018["id"] == i]["type-rue"].iloc[0] for i in uniques]

In [42]:
count = pd.DataFrame({"id":uniques,"count":_count[uniques],"month":months,"road":road,"road_type":road_type,"year":year})
count.reset_index(drop=True, inplace=True)
count.head()

,id,count,month,road,road_type,year
0,SAMSON@2019@6,7,6,SAMSON,BOULEV,2019
1,CHOMEDEY@2019@6,11,6,CHOMEDEY,BOULEV,2019
2,MAURICE-GAUVIN@2019@6,2,6,MAURICE-GAUVIN,RUE,2019
3,RENE-LAENNEC@2019@6,3,6,RENE-LAENNEC,VOIE,2019
4,CARTIER OUEST@2019@6,10,6,CARTIER OUEST,RUE,2019


In [43]:
count.shape

(3155, 6)

## Getting speed limits in the df

---


In [21]:
speed_raw = pd.read_csv("limite-vitesse-2014.csv")
print(speed_raw.shape)
speed_raw.head(5)

(15464, 11)


,WKT,ID_TRC,T_VOIE,L_VOIE,N_VOIE,D_VOIE,LI_GCH,LI_DRT,ODONYME,ODONYME_CO,LIMITE_VIT
0,LINESTRING (-73.6798629184971 45.5559610468134...,508381,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
1,LINESTRING (-73.6806608913008 45.5555611608213...,508380,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
2,LINESTRING (-73.6814206363064 45.5552261180531...,508379,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
3,LINESTRING (-73.6822380588474 45.5549062754073...,508378,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
4,LINESTRING (-73.6830030082074 45.5546018081614...,508377,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50


In [44]:
odonymes = speed_raw["ODONYME"].unique()

In [45]:
lim_speed_by_od = {}
for index,row in speed_raw.iterrows():
    if row[8] not in lim_speed_by_od.keys():
        lim_speed_by_od[row[8]] = row[-1]

In [46]:
set(lim_speed_by_od.keys()) == set(odonymes)

True

In [47]:
check = []
welp = []
matches_dict = {}
for i in rues:
    for j in odonymes:
        if i.lower() in j.lower():
            check.append(True)
            welp.append(True)
            matches_dict[i] = lim_speed_by_od[j]
            break
    check.append(False)

In [48]:
count["speed_limit"] = count["road"].map(matches_dict)
count["speed_limit"] = count["speed_limit"].fillna(0)

In [49]:
count.head()

,id,count,month,road,road_type,year,speed_limit
0,SAMSON@2019@6,7,6,SAMSON,BOULEV,2019,0.0
1,CHOMEDEY@2019@6,11,6,CHOMEDEY,BOULEV,2019,0.0
2,MAURICE-GAUVIN@2019@6,2,6,MAURICE-GAUVIN,RUE,2019,50.0
3,RENE-LAENNEC@2019@6,3,6,RENE-LAENNEC,VOIE,2019,0.0
4,CARTIER OUEST@2019@6,10,6,CARTIER OUEST,RUE,2019,0.0


In [28]:
count = count.dropna()

In [29]:
count

,id,count,month,road,road_type,year,speed_limit
0,SAMSON@2019@6,7,6,SAMSON,BOULEV,2019,0.0
1,CHOMEDEY@2019@6,11,6,CHOMEDEY,BOULEV,2019,0.0
2,MAURICE-GAUVIN@2019@6,2,6,MAURICE-GAUVIN,RUE,2019,50.0
3,RENE-LAENNEC@2019@6,3,6,RENE-LAENNEC,VOIE,2019,0.0
4,CARTIER OUEST@2019@6,10,6,CARTIER OUEST,RUE,2019,0.0
...,...,...,...,...,...,...,...
3150,BELLEROSE EST@2015@1,2,1,BELLEROSE EST,BOULEV,2015,50.0
3151,DE BERRI@2015@1,2,1,DE BERRI,RUE,2015,40.0
3152,MARIAN@2015@1,1,1,MARIAN,RUE,2015,40.0
3153,CHOMEDEY A-13@2015@1,3,1,CHOMEDEY A-13,VOIE,2015,0.0


## Add weather data
---
The weather data was compiled in another notebook so we only have to match with the right months and years.

In [30]:
weather_raw = pd.read_csv("weather_compilation.csv")
weather_raw.head(5)
weather_raw["month"] = weather_raw["month"].astype(int)
weather_raw["year"] = weather_raw["year"].astype(int)

In [31]:
weather_2018 = weather_raw[weather_raw["year"].isin([2015,2016,2017,2018,2019])]
weather_2018["month_year"] = weather_2018["month"].astype(str) + "_" +weather_2018["year"].astype(str)
weather_2018

,Unnamed: 0,year,month,temp,rain,snow,month_year
0,0,2017,1,-4.5470,1.819,8.4200,1_2017
1,1,2017,2,-3.6970,3.188,13.0000,2_2017
2,2,2017,3,-4.0300,2.666,12.2600,3_2017
3,3,2017,4,7.9770,5.094,1.1540,4_2017
4,4,2017,5,13.3100,3.674,0.0000,5_2017
5,5,2017,6,18.8100,3.477,0.0000,6_2017
6,6,2017,7,20.8800,4.035,0.0000,7_2017
7,7,2017,8,19.9400,2.770,0.0000,8_2017
8,8,2017,9,18.5000,1.613,0.0000,9_2017
9,9,2017,10,13.3300,3.742,0.0000,10_2017


In [32]:
count["month_year"] = count["month"].astype(str) +  "_" + count["year"].astype(str)

In [33]:
count

,id,count,month,road,road_type,year,speed_limit,month_year
0,SAMSON@2019@6,7,6,SAMSON,BOULEV,2019,0.0,6_2019
1,CHOMEDEY@2019@6,11,6,CHOMEDEY,BOULEV,2019,0.0,6_2019
2,MAURICE-GAUVIN@2019@6,2,6,MAURICE-GAUVIN,RUE,2019,50.0,6_2019
3,RENE-LAENNEC@2019@6,3,6,RENE-LAENNEC,VOIE,2019,0.0,6_2019
4,CARTIER OUEST@2019@6,10,6,CARTIER OUEST,RUE,2019,0.0,6_2019
...,...,...,...,...,...,...,...,...
3150,BELLEROSE EST@2015@1,2,1,BELLEROSE EST,BOULEV,2015,50.0,1_2015
3151,DE BERRI@2015@1,2,1,DE BERRI,RUE,2015,40.0,1_2015
3152,MARIAN@2015@1,1,1,MARIAN,RUE,2015,40.0,1_2015
3153,CHOMEDEY A-13@2015@1,3,1,CHOMEDEY A-13,VOIE,2015,0.0,1_2015


In [34]:
for i in ["temp","rain","snow"]:
    d = {i:j for i,j in zip(weather_2018["month_year"],weather_2018[i])}
    count[i] = count["month_year"].map(d)

In [35]:
count

,id,count,month,road,road_type,year,speed_limit,month_year,temp,rain,snow
0,SAMSON@2019@6,7,6,SAMSON,BOULEV,2019,0.0,6_2019,18.22,2.693,0.0
1,CHOMEDEY@2019@6,11,6,CHOMEDEY,BOULEV,2019,0.0,6_2019,18.22,2.693,0.0
2,MAURICE-GAUVIN@2019@6,2,6,MAURICE-GAUVIN,RUE,2019,50.0,6_2019,18.22,2.693,0.0
3,RENE-LAENNEC@2019@6,3,6,RENE-LAENNEC,VOIE,2019,0.0,6_2019,18.22,2.693,0.0
4,CARTIER OUEST@2019@6,10,6,CARTIER OUEST,RUE,2019,0.0,6_2019,18.22,2.693,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3150,BELLEROSE EST@2015@1,2,1,BELLEROSE EST,BOULEV,2015,50.0,1_2015,NaN,NaN,NaN
3151,DE BERRI@2015@1,2,1,DE BERRI,RUE,2015,40.0,1_2015,NaN,NaN,NaN
3152,MARIAN@2015@1,1,1,MARIAN,RUE,2015,40.0,1_2015,NaN,NaN,NaN
3153,CHOMEDEY A-13@2015@1,3,1,CHOMEDEY A-13,VOIE,2015,0.0,1_2015,NaN,NaN,NaN


In [36]:
count = count.dropna()

In [37]:
count.to_csv("features_label2.csv")

In [38]:
len(count["speed_limit"].unique())

6